# Webscrape IRCC

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

### Set the URL you want to webscrape from

In [ ]:
CND_SITE = 'https://www.canada.ca'
IRCC_SUFFIX='/en/immigration-refugees-citizenship/'
filter = re.compile('^' + IRCC_SUFFIX + '.*')
url = CND_SITE + IRCC_SUFFIX

### Connect to the URL

In [ ]:
response = requests.get(url)

In [ ]:
response #200 means it went through

<Response [200]>

### Parse HTML and save to BeautifulSoup object

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")

## To download the whole data set, let's do a for loop through all a tags

In [ ]:
count = 10000
for ahref in [a['href'] for a in soup.findAll('a', href=filter)]: #'a' tags are for links
  download_url = CND_SITE + ahref
  count += 1
  urllib.request.urlretrieve(download_url,'./file-'+str(count)) 
  time.sleep(1)
